# Wrangling OpenStreetMap Data 

## Goals and objectives: 

Gain proficiency in data wrangling using OpenStreetMap data.   
- Audit data and devise plan to ensure validity, accuracy, completeness, consistency, and uniformity of data. 
- Create an sql database with appropriate schema, populate tables via python interface (sqlite3).  
- Create functional, readable python code to properly manipulate data.  
- Provide clear documentation 
- Enable statistical analysis of cleaned data 
- Obtain useful information and insight from dataset regarding subject matter.  

## Data Acquisition 

**Data source:** 
The assignment provided data sources that are part of the OpenStreetMap project.  Custom extracts of raw OSM (XML) data were obtained from [mapzen](https://mapzen.com).  

**Subject:** 
The chosen subject matter is the State of Hawai'i. The dataset of the state of Hawai'i (excluding Kauaii) was 158 MB. For efficient auditing (and debugging), smaller samples were used (Island of Hawai'i dataset-- 67.9 MB).   

**'Methods and materials':** 
Python programming language 2.7, IPython (Jupyter) notebook, database schema, Python libraries/packages below: 

In [1]:
OSMFILE = "SofHexK.osm"
DBFILE = "SofHawaii"
# SCHEMA = schema.schema (schema for .csv files)

import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict
from bs4 import BeautifulSoup
import urllib 
from geopy.geocoders import Nominatim
import csv
import codecs
import re
import cerberus
import schema
import sqlite3 
import io

### General description: 

"[OpenStreetMap](https://www.openstreetmap.org/about) is built by a community of mappers that contribute and maintain data about roads, trails, `cafés`, railway stations, and much more, all over the world."

Since the OpenStreetMap is an open source project, human error is likely the main source of error-- namely, inconsistencies in data entry.  The [OSM XML wiki](http://wiki.openstreetmap.org/wiki/OSM_XML) provides documentation for the data.  

The OSM XML provides the framework for `elements` that represent physical features on the map.  `Elements` consist of `nodes`, `ways`, and `relations` (http://wiki.openstreetmap.org/wiki/Elements).  Each instance of `elements` is provided in 'blocks'-- XML elements that enclose tags with key / value attributes.  Documentation describes ['certainties and uncertainties'](http://wiki.openstreetmap.org/wiki/OSM_XML#Certainties_and_Uncertainties) of a given dataset.  


## Data audit 

Data quality is assessed to verify assumptions about the type, shape and value of the data.  Errors and outliers are identified, and missing values are accounted for.  Measures of validity, accuracy, completeness, consistency and uniformity describe the quality of data.  

In [2]:
# Identify and tally the number of each tag in the OSM XML dataset.  Uses ET.fromstring()-- 
# get an idea of performance.

# import xml.etree.cElementTree as ET
# import pprint
# from collections import defaultdict

def count_tags(filename):
        with open(filename, 'r') as f:
            data = f.read()
        tree = ET.fromstring(data)
        d1 = {}
        for el in tree.iter('*'): 
            d1[el.tag] = 0 
        for el in tree.iter('*'): 
            d1[el.tag] +=1 
        return d1


In [3]:
%%timeit -r 1 -n 1
tags = count_tags(OSMFILE)
print tags.keys()

['node', 'nd', 'bounds', 'member', 'tag', 'osm', 'way', 'relation']
1 loop, best of 1: 14.4 s per loop


**'node', 'nd', 'bounds', 'member', 'tag', 'osm', 'way', 'relation'** constitute the xml elements in the OSM dataset.  

Of these, 'way', 'node', 'relation' are osm 'elements'-- the basic components of OpenStreetMap's data model.  

'osm' is the xml element that encloses the entire osm data structure in the the .osm file. 

'bounds' contains attributes that define the boundary coordinates of the map.

'member' is an xml element under the osm element 'relation' (namely the 'multiplygon' relation) that is used to describe how thw 'way's in the 'relation' are related.  

'tag's exist as children of the element trees, fleshing out the details of the element.  

'nd's are tags under 'way's that reference the 'nodes' that make up the 'way's.  

In [4]:
tags = count_tags(OSMFILE)

In [5]:
print tags

{'node': 765029, 'nd': 884680, 'bounds': 1, 'member': 4828, 'tag': 239102, 'osm': 1, 'way': 61483, 'relation': 982}


### Data Validity

In the "State of Hawaii" dataset, ther are 765029 'node', 61483 'way', 982 'relation' elements (i.e. instances thereof).    In the Island of Hawaii OSM dataset, there are 335674 instances of 'node', 25136 instances of 'way', and 415 instances of '`relation`' elements (assessed previously).  The dataset appears to be valid, with no unidentifiable elements or tags.  A validator will be used in the subequent sql database intake. 

#### Verification / validation scheme of the tags in the osm dataset: 

osm: verify that there is only one.  
bounds: verify coordinates of bounds.  
member: all members are ways?  
nds: all nodes in dataset?  
tags: what features are represented?  

As expected, there is only one `osm` element and one `bounds` element.  

In [6]:
# These are functions used in verifying osm element data 

# view the tags
def view_attribs(tagstring):
    for _, element in ET.iterparse(OSMFILE): 
        if element.tag == tagstring: 
            pprint.pprint(element.attrib)
    
# get the tag
def get_attrib(tagstring):
    for _, element in ET.iterparse(OSMFILE): 
        if element.tag == tagstring: 
            return element.attrib

# get the set of all values of a given keyvalue in a given tag
def get_allof_attrib(tagstring, keystring): 
    attrib_set = set()
    for _, element in ET.iterparse(OSMFILE): 
        if element.tag == tagstring and keystring in element.attrib.keys(): 
            attrib_set.add(element.attrib[keystring])
    return attrib_set

# get the set of all values of a given keyvalue of a given child tag of a tag  
def get_allof_childattrib(tagstring, childtagstring, childkeystring): 
    attrib_set = set() 
    for _, element in ET.iterparse(OSMFILE): 
        if element.tag == tagstring: 
            for a in element.findall(childtagstring): 
                attrib_set.add(a.get(childkeystring))
    return attrib_set

In [7]:
# 
bounds = get_attrib('bounds')
print bounds

{'minlat': '18.6982854', 'maxlon': '-154.6325683', 'minlon': '-158.4338378', 'maxlat': '21.8411047'}


The `bounds` attributes provide the coordinates for the map.  Using the '`geopy`' package, we can double check that the coordinates correspond to the actual physical location.  

In [8]:
# verify location of coordinates using geopy package (https://pypi.python.org/pypi/geopy#downloads)
# from geopy.geocoders import Nominatim
geolocator = Nominatim()
minloc = geolocator.reverse("{},{}".format(bounds['minlat'], bounds['minlon']))
maxloc = geolocator.reverse("{},{}".format(bounds['maxlat'], bounds['maxlon']))
print maxloc
#print minloc # returns TypeError: __str__ returned non-string (type NoneType)-- coordinates unlabeled. 


96816


'96816' is a zipcode of Honolulu.  

All constituents of the 'members' and 'nd' osm tags must be an osm element ('ways' or 'nodes').  

In [9]:
# all 'way' ids: 
way_ids = get_allof_attrib('way','id')

# all 'relation' ids: 
relation_ids = get_allof_attrib('relation', 'id')

# all 'member' ids in 'relation': 
relation_member_refs = get_allof_childattrib('relation', 'member', 'ref')

# etc. 
way_changesets = get_allof_attrib('way', 'changeset')

node_ids = get_allof_attrib('node', 'id')

way_nd_refs = get_allof_childattrib('way', 'nd', 'ref')

node_nd_refs = get_allof_childattrib('node', 'nd', 'ref')

In [10]:
# how many of set one is /isn't in set2?
def what_in_what(set1, set2):
    is_in = 0 
    not_in = 0 
    for id in set1: 
        if id not in set2: 
#            print "{} not in {}".format(id, 'way_ids')
            not_in+=1
        elif id in set2: 
#            print "{} in {}".format(id, 'way_ids')
            is_in+=1
    print 'not_in: {}, is_in: {}'.format(not_in, is_in)
    
# get list of those not in set2
def listof_notin(set1, set2):
    is_in = 0 
    not_in = 0 
    notinlist = []
    for id in set1: 
        if id not in set2: 
#            print "{} not in {}".format(id, 'way_ids')
            not_in+=1
            notinlist.append(id)
        elif id in set2: 
#            print "{} in {}".format(id, 'way_ids')
            is_in+=1
    return notinlist

In [11]:
what_in_what(relation_member_refs, way_ids)

not_in: 317, is_in: 3108


In [12]:
what_in_what(relation_member_refs, node_ids)

not_in: 3119, is_in: 306


In [13]:
what_in_what(relation_member_refs, way_ids.union(node_ids))

not_in: 11, is_in: 3414


In [14]:
listof_notin(relation_member_refs, way_ids.union(node_ids))

['4090361',
 '4090378',
 '4090377',
 '3697279',
 '3697274',
 '168936',
 '168937',
 '166563',
 '4090539',
 '4090538',
 '4089706']

There are 11 'member' element that is neither 'node' nor 'way'.  This can be flagged for analysis later.  

In [15]:
what_in_what(relation_member_refs, way_ids.union(node_ids).union(way_nd_refs).union(node_nd_refs).union(relation_ids))

not_in: 0, is_in: 3425


In fact, the 11 'member's were themselves 'relation's.  

In [16]:
what_in_what(relation_member_refs, relation_ids)

not_in: 3414, is_in: 11


All 'way node's are a subset of nodes.  

In [17]:
what_in_what(way_nd_refs, node_ids)

not_in: 0, is_in: 759598


In [18]:
nd_refs = get_allof_attrib('nd', 'ref')
len(set(nd_refs))

759598

In [19]:
def get_value(element, kitem):
    return element.attrib[kitem]

def get_allelement_attrib(kitem):
    k = set()
    for _, element in ET.iterparse(OSMFILE):
        if  kitem in element.attrib.keys():
            k.add(get_value(element, kitem))
    return k

users = get_allelement_attrib('uid')
len(users)

731

There are 731 unique contributors who have worked on this map.  

### Consistency and Completeness

**Consistency:**'k' (key) and 'v' (value) attributes of tags describe the enclosing element.  Both keys and values are 'free format text fields' describing the features of a map object.  While this confers user flexibility for the open data project, it does not bode well for data consistency, as the same physical object on the map can be variously described in multiple observations.  

**Completeness:**  An open project dataset is characteristically incomplete.  In addition, OSM's 'free tagging system' provides an unlimited number of ways a map feature can be described.  The 'degree of incompleteness' of a dataset can be characterized to understand what aspects of the map needs particular attention.  


In [20]:
# ### Generates manageabe sample data by aggregating every kth element.  (Code adapted from Udacity.)

# # import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

# OSM_FILE = "Hawaii.osm"  # input dataset file
# SAMPLE_FILE = "sample.osm" # output sample file

# k = 10 # Parameter: take every k-th top level element

# def get_element(osm_file, tags=('node', 'way', 'relation')):
#     context = iter(ET.iterparse(osm_file, events=('start', 'end')))
#     _, root = next(context)
#     for event, elem in context:
#         if event == 'end' and elem.tag in tags:
#             yield elem  #`yield` returns a generator (an iterable object, iterating one by one)
#             root.clear()


# with open(SAMPLE_FILE, 'wb') as output:
#     output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
#     output.write('<osm>\n  ')

#     # Write every kth top level element
#     for i, element in enumerate(get_element(OSM_FILE)):
#         if i % k == 0:
#             output.write(ET.tostring(element, encoding='utf-8'))

#     output.write('</osm>')

### Scraping key / value standards and map features

In order to make sense of the key/value string attributes of tag elements, the "Map Features" section of the Wiki was scraped to extract the commonly used keys and values.  These comprise the informal standards for categorizing map features. Assessing the data with this information is one way to gain insight into the quality of the data.  

In [21]:
# from bs4 import BeautifulSoup
# import urllib 

# access the site and obtain BS object 
wikisite = 'http://wiki.openstreetmap.org/wiki/Map_Features'
r = urllib.urlopen(wikisite).read() 
soup = BeautifulSoup(r, 'html.parser')

In [22]:
# Displays the wiki 
# from IPython.display import HTML
# HTML(wikisite)

In [23]:
# Find all the wiki tables.  
tables = soup.find_all("table", class_='wikitable')

In [24]:
len(tables)

29

In [25]:
# list of key values that are physical map features 
keys = [table.a['href'] for table in tables]



The features are displayed in 29
tables.  However, the data structure represented by the tables are not consistent.  The 'key' value may represent a map feature, or an attribute of the feature; a 'value' value may be a subcategory of a map feature or an attribute.  Key values not representing map features were manually removed.  

In [26]:
rows = soup.find_all('tr')

In [27]:
features = []
for n, a1 in enumerate(rows): 
    a2 = a1.find_all('td')
    try:
        if a2[0].a['href'] in keys and a2[1].a['title']: 
                values = a2[1].a['title'].split(':')[1]
                if "(page does not exist)" not in values: 
                    features.append(values)                
    except: 
        pass

In [28]:
# Find the elements with the 'key' values, 'values' values that correspond to an osm map feature.  
# Count the occurence of each map feature.  

# from collections import defaultdict 

key_tally = defaultdict(set)
value_tally = defaultdict(set) 
feature_tally = defaultdict(set)

for a in features: 
    key_tally[a.split('=')[0]]=0
    value_tally[a.split('=')[1]]=0
    feature_tally[a] = 0
    
def key_type(element, tally):
    for a in tally: 
        if element.attrib['k'] == a: 
            tally[a] +=  1              
    return tally

def value_type(element, tally): 
    for a in tally: 
        a1 = a.split('=')
        if element.attrib['k'] == a1[0] and element.attrib['v'] == a1[1]: 
            tally[a] += 1 
    return tally

def process_attrib(filename, tally, attrib):
    for _, element in ET.iterparse(filename):
        if element.tag == "tag": 
            if attrib == 'k':
                tally = key_type(element, tally)
            elif attrib == 'v': 
                tally = value_type(element, tally)
    return tally


In [29]:
featuresinHawaii = process_attrib(OSMFILE, feature_tally, 'v')

In [30]:
featuresinHawaii2 = defaultdict(set)
for a in featuresinHawaii: 
    if featuresinHawaii[a] != 0: 
        featuresinHawaii2[a] = featuresinHawaii[a]

In [31]:
featuresinHawaii2

defaultdict(set,
            {u'access=designated': 3,
             u'aeroway=aerodrome': 47,
             u'aeroway=apron': 35,
             u'aeroway=gate': 42,
             u'aeroway=helipad': 144,
             u'aeroway=heliport': 1,
             u'aeroway=navigationaid': 1,
             u'aeroway=runway': 51,
             u'aeroway=taxiway': 303,
             u'aeroway=terminal': 19,
             u'aeroway=windsock': 5,
             u'amenity=atm': 10,
             u'amenity=bank': 44,
             u'amenity=bar': 24,
             u'amenity=bbq': 1,
             u'amenity=bench': 25,
             u'amenity=cafe': 104,
             u'amenity=cinema': 8,
             u'amenity=clinic': 5,
             u'amenity=college': 71,
             u'amenity=courthouse': 1,
             u'amenity=dentist': 3,
             u'amenity=doctors': 2,
             u'amenity=dojo': 1,
             u'amenity=embassy': 1,
             u'amenity=fountain': 8,
             u'amenity=fuel': 69,
           

In [32]:
len(featuresinHawaii2)

247

The above dictionary provides a tally of all the features on this map.  Subsequent analysis with the use of supporting information can delve deeper into characterizing the dataset, and also give insight into the subject matter. 

### Accuracy and uniformity 

Besides the issue of accuracy of the features described in the OSM elements, proper nomenclature of map objects (streets, buildings, etc.) in the Hawaiian language is another dimension to consider.  The Hawaiian language did not exist in written form (except in petroglyph symbols) until the 1820s; in its current use, the Hawaiian written language uses 12 letters of the english alphabet, plus a glottal stop (the 'okina).  The vowels can also have macrons (looks like a hypen on top) that affect its pronunciation.  In practice, many pidgin/creole and colloquial terms exist as it has fluidly absorbed  foreign words.  For these reasons, the Hawaiian language is difficult to formalize.  Nevertheless, the landmarks presumably follow formal nomenclature that gives proper respect to the culture of Hawai'i.  The text data can be assessed for accuracy by comparing with an outside source (an official lexicon), while 'allophones' can be consolidated to allow uniformity in names.  

Textual input of street names can give rise to many variants.  The '`addr`' category in map features has specific sub-fields for respective components of a postal address (street name, number, postal code, etc.).  User omission, typos, miscategorization, abbreviations, etc. are common sources of variability.  

The end word of street names was audited to account for variations.  A typical ending is expected ('street', 'avenue', etc.) for street names.  In Hawai'i, Hawaiian street names typically include the expected ending, but exceptions may exist.  

In [33]:
# Audit street names: surve street name endings 
# Map nomenclature variants in a dictionary ('mapping')
# Correct 

#from collections import defaultdict

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Highway", "Loop", "Mall", "Terrace", "Parkway", "Circle", "Way", "Center"]


mapping = { "St": "Street",
            "St.": "Street", 
            "Ave": "Avenue", 
           "AVE" : "Avenue", 
            "Rd.": "Road", 
           "Rd" : "Road", 
           "Blvd": "Boulevard", 
           "Hwy" : "Highway", 
           "Pkwy" : "Parkway", 
           "highway" : "Highway"
           
            }

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    count = 0
    # parses osm file, 
    #finds "tag" or "way" elements 
    #if k attibute indicates enclosing element tag is street address, 
    #evaluates tag attribute value (which is street name string)
    #
#    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osmfile, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    
                    audit_street_type(street_types, tag.attrib['v'])
#                    print tag.attrib['v']
                    count += 1 
#    osm_file.close()
#    print "street types", street_types
    print 'street count is:', count
    return street_types


def update_name(name, mapping):
    
#    print 'name is:',name.split(' ')[-1], 'mapping is ', mapping
    wordlist = name.split(' ')
    lastbit = wordlist[-1]
    lastlen = len(lastbit)
    rest = name[:-lastlen]
    m = re.search(r'\S{1,4}\.?$', lastbit) 
#    print 'lastbit:',lastbit, m.group()
#    street_types[m.group()].add(name)
    if m.group() in mapping.keys(): 
        name = rest + mapping[m.group()]
#        mapping[lastbit] = mapping[m.group()] 
    
    else: 
        for ex in expected: 
            if unicode(ex.lower) == unicode(lastbit): 
                name = rest + ex 
            elif ex[0].lower ==lastbit[0].lower and (ex[1] == lastbit[1] or lastbit[1] == ex[-1]):
                name = rest + ex
            elif lastbit == 'highway': 
                name = rest + mapping[lastbit]
#                print name

#                mapping[lastbit] = ex 
                
#        for a in 
#    print "mapping:",mapping
    return name

st_types = audit(OSMFILE)
# pprint.pprint(dict(st_types))

for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name, "=>", better_name
print 'corrected name count is:', len(st_types.items())

        


street count is: 973
Lusitania St. => Lusitania Street
Makena Alanui => Makena Alanui
Kapalai Rd => Kapalai Road
Paradise Ala Kai => Paradise Ala Kai
Pali Momi => Pali Momi
Moanalua, Honolulu => Moanalua, Honolulu
Kamehameha Hwy => Kamehameha Highway
Orchid Land Dr => Orchid Land Drive
Kipapa Dr => Kipapa Drive
kanehameha highway => kanehameha Highway
South King => South King
Pi'Ikea Ave. => Pi'Ikea Avenue
South Kamehameha Ave. => South Kamehameha Avenue
101 Pakaula St Kahului 96732 => 101 Pakaula St Kahului 96732
Wainee St => Wainee Street
Ala Pumalu St => Ala Pumalu Street
Lusitania St => Lusitania Street
Milo St => Milo Street
Kalakaua Ave => Kalakaua Avenue
Meheula Pkwy => Meheula Parkway
Pualei Cir, Apt 106 => Pualei Cir, Apt 106
king => king
Wailea Alanui Drive #A23 => Wailea Alanui Drive #A23
Beach Walk => Beach Way
Mauna Lani Drive #410 => Mauna Lani Drive #410
Ala Moana Blvd => Ala Moana Boulevard
Kahakai Blvd => Kahakai Boulevard
HINA AVE => HINA Avenue
corrected name count i

___

There are 973 instances of 'street address'es on this map.  Of these, 22 were flagged for correction into their proper endings.  

A street count of 973 seems to be low for an entire state; this indicates there is much work to be done for the OSM project.  

___

## Data handling: .osm to .csv to sql

As an intermediate step to the creation of an sql database, 'node' and 'ways' element data were extracted from the .osm file and organized into .csv datasets.  A validation process was included to ensure that the data fit the schema. The schema shapes the osm data structure into a normalized table template.  

In [34]:
'''
Import to .csv (Code adapted from Udacity course exercises)
'''
# import csv
# import codecs
# import re
# import xml.etree.cElementTree as ET
# import cerberus
# import schema

OSM_PATH = DBFILE
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"
DB_TABLES = ('nodes','node_tags','ways','way_nodes','way_tags') 
LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position'] 

In [35]:
# import re
# import pprint

# shapes elements into a flattened table
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""
    
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        node_attribs = {
            'id':element.attrib['id'], 
            'user':element.attrib['user'], 
            'uid':element.attrib['uid'], 
            'version': element.attrib['version'],
            'lat': element.attrib['lat'], 
            'lon': element.attrib['lon'], 
            'timestamp': element.attrib['timestamp'], 
            'changeset': element.attrib['changeset']
            } 
        tags = shape_tags(element, tags)
        return {'node': node_attribs, 'node_tags': tags}

    elif element.tag == 'way':
        way_attribs = {
            'user':element.attrib['user'], 
            'uid':element.attrib['uid'], 
            'version': element.attrib['version'],
            'id': element.attrib['id'], 
            'timestamp': element.attrib['timestamp'], 
            'changeset': element.attrib['changeset']
            } 
        tags = shape_tags(element,tags)
        # way nodes 
        for n, n1 in enumerate(element.iter("nd")): 
            nd1 = { 
                'id': element.attrib['id'], 
                'node_id': n1.attrib['ref'], 
                'position': n
            } 
            way_nodes.append(nd1)
        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

# shapes osm element attributes into a flattened structure
def shape_tags(element, tags): 
        try: 
            for c1 in element.iter("tag"): 
                c2= {}
                c2['id'] = element.attrib['id']
                c3 = c1.attrib['k'].split(':') 

                if len(c3) == 1: 
                    c2['key'] = c1.attrib['k'] 
                    c2['type'] = 'regular' 
                    c2['value'] = c1.attrib['v']
                elif len(c3) == 2: 
                    c2['key'] = c3[1] 
                    c2['type'] = c3[0]
                c2['value'] = c1.attrib['v'] 
 
                if len(c3) == 3: 
                    c2['key'] = str(c3[1]+":"+c3[2])
                    c2['type'] = c3[0]

                tags.append(c2) 
        except: 
            print "error in ", c2

        return tags
    
# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

# uses a validator (cerberus) with schema
def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )

# Allows csv module to handle unicode text
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""
    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])





In [36]:
# Process .osm file into .csvs
process_map(OSMFILE, 'True')

The csv library cannot handle unicode input.  Some user-entered text entries (e.g. '`utf8-xe3x83x92xe3x83xad`') were in unicode which raised errors when passed into `csvreader`.  A wrapper for the reader module was available to convert unicode to UTF-8.   


### Database creation and table insertion

The data in .csv files are now inserted into an sql database.  Appropriate tables (one per osm element and respective tags) are created.  

In [37]:
# Create sql table with schema provided. 

# import sqlite3 
with sqlite3.connect(DBFILE) as db: 
    cursor = db.cursor()

# create dictionary of queries, one for each table needed in database, directly matching sql schema  
QUERY = {
    'nodes':
        '''CREATE TABLE nodes (
        id INTEGER PRIMARY KEY NOT NULL,
        lat REAL,
        lon REAL,
        user TEXT,
        uid INTEGER,
        version INTEGER,
        changeset INTEGER,
        timestamp TEXT );''' ,
    'nodes_tags':    
        '''CREATE TABLE nodes_tags (
        id INTEGER,
        key TEXT,
        value TEXT,
        type TEXT,
        FOREIGN KEY (id) REFERENCES nodes(id) ); ''', 
    'ways':    
        '''CREATE TABLE ways (
        id INTEGER PRIMARY KEY NOT NULL,
        user TEXT,
        uid INTEGER,
        version TEXT,
        changeset INTEGER,
        timestamp TEXT ); ''', 
    'ways_tags':    
        '''CREATE TABLE ways_tags (
        id INTEGER NOT NULL,
        key TEXT NOT NULL,
        value TEXT NOT NULL,
        type TEXT,
        FOREIGN KEY (id) REFERENCES ways(id) ); ''', 
    'ways_nodes':
        '''CREATE TABLE ways_nodes (
        id INTEGER NOT NULL,
        node_id INTEGER NOT NULL,
        position INTEGER NOT NULL,
        FOREIGN KEY (id) REFERENCES ways(id),
        FOREIGN KEY (node_id) REFERENCES nodes(id)
        );'''
}

In [38]:
# 'Helper files for data import to database

# import csv 
# from pprint import pprint
# import io

# change unicode bytestrings to utf-8
class UTF8Recoder:
    """
    Iterator that reads an encoded stream and reencodes the input to UTF-8
    """
    def __init__(self, f, encoding):
        self.reader = codecs.getreader(encoding)(f)

    def __iter__(self):
        return self

    def next(self):
        return self.reader.next().encode("utf-8")
class UnicodeReader:
    """
    A CSV reader which will iterate over lines in the CSV file "f",
    which is encoded in the given encoding.
    """

    def __init__(self, f, dialect=csv.excel, encoding="utf-8", **kwds):
        f = UTF8Recoder(f, encoding)
        self.reader = csv.reader(f, dialect=dialect, **kwds)

    def next(self):
        row = self.reader.next()
        return [unicode(s, "utf-8") for s in row]

    def __iter__(self):
        return self

In [39]:
# Next iteration: node_tags.csv to node_tags table 
# create a function to transfer data from .csv to sql database table
## extract rows of data from .csv to tuples, rendering all values into unicode 

def csv_to_tuple(csvfile):
    with open(csvfile,'rb') as csvf: 
        csvReader = UnicodeReader(csvf)
        csvd = [tuple(a1) for a1 in csvReader]
        header = csvd[0]
        print 'csv file header: ', header
        return csvd

# takes list of tuples (tlist, extracted from .csv) and inserts into sql table in database
def tuples_to_table(tlist, table, db): 
    db1 = sqlite3.connect(db)
    cursor1 = db1.cursor()    
    vname_ = tuple(a.encode('utf-8') for a in tlist[0]) # sql variable name must be a tuple in UTF_8 encoding
    print 'table columns: ', vname_
    cursor1.executemany(
    '''INSERT INTO {}{} VALUES ({}?);'''.format(table, 
                                                 vname_, 
                                                 '?,'*(len(tlist[0])-1)), 
        tlist[1:])
    db1.commit() 
    db1.close()


In [40]:
# Next iteration: ways.csv to ways table 
# create function that takes data from .csv to table from start to finish  

def csv_to_table(csvfile, table, dbfile): 
    
    tdata = csv_to_tuple(csvfile) 
#    print 'tdata[:2]', tdata[:2]
    tuples_to_table(tdata, table, dbfile)    

#csv_to_table(CSVFILE, TABLE, DBFILE)

In [41]:
# Next iteration: ways_tags to database 
# Create function that creates database and necessary tables, takes data from .csv file to table  

def csv_to_sql(csvfile, table, database, sqlquery): 
    db = sqlite3.connect(database)
    cursor = db.cursor()
    
    # See if table exists: if true, drop and create new
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='{}';".format(table))  
    rows = cursor.fetchall() 
    if rows: 
        cursor.execute("DROP TABLE {}".format(table))
        print "table dropped:", cursor.fetchall()
    print 'Creating table', table
    cursor.execute(sqlquery)    
    db.commit()

    # Transfer data from .csv to table
    csv_to_table(csvfile, table, database)
    print 'final call:', cursor.fetchall()
    db.close()

#csv_to_table(CSVFILE, TABLE, DBFILE)

In [42]:
# Create database tables.

# DBFILE = "SofHawaii" 
CSVFILES = ['nodes.csv', 'nodes_tags.csv', 'ways.csv', 'ways_nodes.csv', 'ways_tags.csv']
DBTABLES = ['nodes', 'nodes_tags', 'ways', 'ways_nodes', 'ways_tags']
QUERY = QUERY

for n, csvfile in enumerate(CSVFILES): 
    table = DBTABLES[n]
    csv_to_sql(csvfile, table, DBFILE, QUERY[table])

table dropped: []
Creating table nodes
csv file header:  (u'id', u'lat', u'lon', u'user', u'uid', u'version', u'changeset', u'timestamp')
table columns:  ('id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp')
final call: []
table dropped: []
Creating table nodes_tags
csv file header:  (u'id', u'key', u'value', u'type')
table columns:  ('id', 'key', 'value', 'type')
final call: []
table dropped: []
Creating table ways
csv file header:  (u'id', u'user', u'uid', u'version', u'changeset', u'timestamp')
table columns:  ('id', 'user', 'uid', 'version', 'changeset', 'timestamp')
final call: []
table dropped: []
Creating table ways_nodes
csv file header:  (u'id', u'node_id', u'position')
table columns:  ('id', 'node_id', 'position')
final call: []
table dropped: []
Creating table ways_tags
csv file header:  (u'id', u'key', u'value', u'type')
table columns:  ('id', 'key', 'value', 'type')
final call: []


## Basic EDA 
Study the top 5 'node' contributors to the dataset. 

In [43]:
# 

def do_sql(database): 
    db = sqlite3.connect(database)
    c = db.cursor()

    c.execute(query) 
    rows = c.fetchall() 
#     for r in rows: 
#         print r
    db.close()
    return rows

These are the top 5 contributors: 

In [44]:
query = '''
SELECT user, count(*) as num
FROM nodes 
GROUP BY user
ORDER BY num DESC
limit 5;
'''
output = do_sql(DBFILE)
for r in output: 
    print r

(u'Tom_Holland', 311519)
(u'bdiscoe', 114411)
(u'ksamples', 100433)
(u'ikiya', 26288)
(u'Chris Lawrence', 18810)


In [45]:
top5_users = [a[0] for a in output]

Exploratory questions: 
- what features have they mapped? 
- how are the locations distributed on the map? 

In [58]:
# Find the top 20 osm element tags for each user
query = ''' 
SELECT nodes_tags.key, nodes_tags.value, nodes_tags.type, nodes.id, COUNT(*) AS num
FROM nodes_tags, nodes
WHERE nodes.id = nodes_tags.id
AND user = 'Tom_Holland'
GROUP BY nodes_tags.key
ORDER BY num DESC 
LIMIT 20;
'''
output = do_sql(DBFILE)
for r in output: 
    print r

(u'power', u'pole', u'regular', 4360423644, 1102)
(u'highway', u'turning_circle', u'regular', 4394324249, 870)
(u'name', u'Two Step Beach', u'regular', 4393822339, 702)
(u'natural', u'tree', u'regular', 4350509140, 460)
(u'ele', u'28', u'regular', 4352202674, 423)
(u'place', u'locality', u'regular', 4393822339, 229)
(u'state', u'HI', u'addr', 4352202674, 223)
(u'feature_id', u'1965020', u'gnis', 4352202674, 222)
(u'county_name', u'Hawaii', u'gnis', 4352202674, 219)
(u'created', u'02/06/1981', u'gnis', 2409055920, 196)
(u'feature_type', u'Bay', u'gnis', 2409055920, 196)
(u'import_uuid', u'57871b70-0100-4405-bb30-88b2e001a944', u'gnis', 4352202674, 181)
(u'is_in', u'Hawaii,Hawaii,Hawaii,HI,USA', u'regular', 2039216241, 165)
(u'Class', u'Populated Place', u'gnis', 2039216241, 158)
(u'County', u'Hawaii', u'gnis', 2039216241, 158)
(u'County_num', u'001', u'gnis', 2039216241, 158)
(u'ST_alpha', u'HI', u'gnis', 2039216241, 158)
(u'ST_num', u'15', u'gnis', 2039216241, 158)
(u'id', u'1852577', 

In [50]:
query = ''' 
SELECT nodes_tags.key, nodes_tags.value, nodes.id, nodes_tags.type, COUNT(*) AS num
FROM nodes_tags, nodes
WHERE nodes.id = nodes_tags.id
AND user = 'bdiscoe'
GROUP BY key
ORDER BY num DESC 
LIMIT 20;
'''
output = do_sql(DBFILE)
for r in output: 
    print r

(u'highway', u'turning_circle', 3998481317, u'regular', 411)
(u'power', u'pole', 3993959704, u'regular', 139)
(u'barrier', u'gate', 3994059217, u'regular', 45)
(u'name', u'Uncle Robert\u02bbs', 2906628101, u'regular', 37)
(u'source', u'wind', 2967156335, u'generator', 29)
(u'ele', u'438', 368391640, u'regular', 22)
(u'place', u'farm', 2399899987, u'regular', 22)
(u'is_in', u'USA, Hawaii', 1635717703, u'regular', 21)
(u'import_uuid', u'57871b70-0100-4405-bb30-88b2e001a944', 367803676, u'gnis', 20)
(u'Class', u'Populated Place', 150921304, u'gnis', 19)
(u'County', u'Hawaii', 150921304, u'gnis', 19)
(u'County_num', u'001', 150921304, u'gnis', 19)
(u'ST_alpha', u'HI', 150921304, u'gnis', 19)
(u'ST_num', u'15', 150921304, u'gnis', 19)
(u'id', u'361505', 150921304, u'gnis', 19)
(u'amenity', u'parking', 2933892351, u'regular', 15)
(u'state', u'Hawaii', 1635717703, u'is_in', 4)
(u'ford', u'yes', 2398141212, u'regular', 3)
(u'tourism', u'attraction', 2065645543, u'regular', 3)
(u'CLASSFP', u'U1

In [51]:
query = ''' 
SELECT nodes_tags.key, nodes_tags.value, nodes.id, nodes_tags.type, COUNT(*) AS num
FROM nodes_tags, nodes
WHERE nodes.id = nodes_tags.id
AND user = 'ikiya'
GROUP BY key
ORDER BY num DESC 
LIMIT 20;
'''
output = do_sql(DBFILE)
for r in output: 
    print r

(u'highway', u'turning_circle', 2034125148, u'regular', 167)
(u'source', u'Bing', 2034125148, u'regular', 162)
(u'barrier', u'lift_gate', 1690079522, u'regular', 1)


In [55]:
query = ''' 
SELECT nodes_tags.key, nodes_tags.value, nodes.id, nodes_tags.type, COUNT(*) AS num
FROM nodes_tags, nodes
WHERE nodes.id = nodes_tags.id
AND user = 'ikiya'
GROUP BY key
ORDER BY num DESC 
LIMIT 20;
'''
output = do_sql(DBFILE)
for r in output: 
    print r

(u'highway', u'turning_circle', 2034125148, u'regular', 167)
(u'source', u'Bing', 2034125148, u'regular', 162)
(u'barrier', u'lift_gate', 1690079522, u'regular', 1)


In [57]:
query = ''' 
SELECT nodes_tags.key, nodes_tags.value, nodes.id, nodes_tags.type, COUNT(*) AS num
FROM nodes_tags, nodes
WHERE nodes.id = nodes_tags.id
AND user = 'Chris_Lawrence'
GROUP BY key
ORDER BY num DESC 
LIMIT 20;
'''
output = do_sql(DBFILE)
for r in output: 
    print r

In [53]:
query = ''' 
SELECT id, COUNT(*) AS num
FROM nodes
WHERE user = 'Chris Lawrence'
GROUP BY id
ORDER BY num DESC 
LIMIT 20;
'''
output = do_sql(DBFILE)
for r in output: 
    print r

(384258507, 1)
(384258508, 1)
(384258521, 1)
(384258522, 1)
(384258523, 1)
(384258524, 1)
(384258525, 1)
(384258528, 1)
(384258529, 1)
(384258530, 1)
(384258531, 1)
(384258532, 1)
(384258533, 1)
(384258534, 1)
(384258535, 1)
(384258536, 1)
(384258537, 1)
(384258538, 1)
(384258539, 1)
(384258540, 1)


A simple tally and observation of the most frequently tagged node attributes for the top 5 players show different behavior.  With my limited knowledge of OSM: the top contributor makes frequent use of gnis data, while the second likely makes use of gps.  The third is most likey from a Nokia device with GPS.  The last two have no node tag information; it may be a device-automated entry or an outdated contribution.  

In [59]:
# Retrieve all osm element tags for user
query = ''' 
SELECT nodes_tags.key, nodes_tags.value, nodes_tags.type, nodes.id, COUNT(*) AS num
FROM nodes_tags, nodes
WHERE nodes.id = nodes_tags.id
AND user = 'Tom_Holland'
GROUP BY nodes_tags.key
ORDER BY num DESC 
;
'''
output = do_sql(DBFILE) 

In [65]:
# use featuresinHawaii2 obtained above, split features into list of key, value pairs
FH = []
for a1 in featuresinHawaii2.items(): 
    try: FH.append(a1[0].split('=')) 
    except: print 'ommited ', a1
        

ommited  (0, set([]))


In [74]:
# map features tagged by user 'Tom_Holland'

for a1 in output: 
    for b1 in FH: 
        if a1[0] == b1[0] and a1[1] == b1[1]: 
            print a1 
            
    

(u'power', u'pole', u'regular', 4360423644, 1102)
(u'natural', u'tree', u'regular', 4350509140, 460)
(u'place', u'locality', u'regular', 4393822339, 229)
(u'barrier', u'gate', u'regular', 4398466564, 147)
(u'tourism', u'information', u'regular', 4392870883, 66)
(u'leisure', u'slipway', u'regular', 4393822642, 48)
(u'sport', u'swimming', u'regular', 2560671485, 16)
(u'landuse', u'military', u'regular', 2157554429, 11)
(u'waterway', u'waterfall', u'regular', 2268400428, 9)
(u'historic', u'monument', u'regular', 2471874267, 3)
(u'emergency', u'defibrillator', u'regular', 2482094584, 1)


In [75]:
# Retrieve all osm element tags for user
query = ''' 
SELECT nodes_tags.key, nodes_tags.value, nodes_tags.type, nodes.id, COUNT(*) AS num
FROM nodes_tags, nodes
WHERE nodes.id = nodes_tags.id
AND user = 'bdiscoe'
GROUP BY nodes_tags.key
ORDER BY num DESC 
;
'''
output = do_sql(DBFILE) 

In [76]:
# map features tagged by user 'bdiscoe'

for a1 in output: 
    for b1 in FH: 
        if a1[0] == b1[0] and a1[1] == b1[1]: 
            print a1 
            
    

(u'power', u'pole', u'regular', 3993959704, 139)
(u'barrier', u'gate', u'regular', 3994059217, 45)
(u'place', u'farm', u'regular', 2399899987, 22)
(u'amenity', u'parking', u'regular', 2933892351, 15)
(u'tourism', u'attraction', u'regular', 2065645543, 3)
(u'aeroway', u'aerodrome', u'regular', 368391640, 1)


Simple observations of data obtained by 'Tom_Holland' and 'bdiscoe' provide insight into who they are.  'Tom_Holland' is likely military personnel residing in Hawaii, while 'bdiscoe' was likely to be a tourist visiting the island.  

Further explorations into contributor behavior can provide locations of tags using geopy.  

In [86]:
# Retrieve all osm element tags for user
query = ''' 
SELECT nodes_tags.key, nodes_tags.value, nodes.lat, nodes.lon, nodes.timestamp, COUNT(*) AS num
FROM nodes_tags, nodes
WHERE nodes.id = nodes_tags.id
AND user = 'Tom_Holland'
GROUP BY nodes_tags.key
ORDER BY num DESC 
;
'''
output = do_sql(DBFILE) 

In [90]:
for a1 in output[:20]: 
    print unicode(geolocator.reverse("{},{}".format(a1[2], a1[3]))), a1[4]
    

Alulike Trail, Waikii, Kohala, Hawaii, United States of America 2016-08-21T18:17:59Z
Lower Napo'opo'o Road, Honaunau-Napoopoo CDP, Kau, Hawaii, 96704, United States of America 2016-09-10T03:24:22Z
Keoneele Cove, Honaunau Beach Road, Honaunau-Napoopoo CDP, Kau, Hawaii, 96704, United States of America 2016-09-09T18:12:20Z
98, Laimana Street, Pu‘u‘eo, Hilo CDP, North Hilo, Hawaii, 96720, United States of America 2016-08-15T04:01:55Z
College Hall, 200, West Kawili Street, Waiākea, Hilo CDP, South Hilo, Hawaii, 96720, United States of America 2016-08-16T07:30:31Z
Keoneele Cove, Honaunau Beach Road, Honaunau-Napoopoo CDP, Kau, Hawaii, 96704, United States of America 2016-09-09T18:12:20Z
College Hall, 200, West Kawili Street, Waiākea, Hilo CDP, South Hilo, Hawaii, 96720, United States of America 2016-08-16T07:30:31Z
College Hall, 200, West Kawili Street, Waiākea, Hilo CDP, South Hilo, Hawaii, 96720, United States of America 2016-08-16T07:30:31Z
College Hall, 200, West Kawili Street, Waiākea, 

The results above show that the tags obtained by 'Tom_Holland' were obtained over several years, and are all located in the Island of Hawai'i.  


## Questions 



### Further questions about Hawaii 


Submission document includes one or more additional suggestions for improving and analyzing the data.  
Many sites in Hawai'i have official names but more frequently used colloquial terms.  A separate feature tag designation for a single official name and (multiple) unofficial names might clarify the nomenclature.  It would still be up to the community to observe and enforce such distinctions, replete with the correct lexical represetation. ISO 639 is the designated Hawaiian character set.  

The following references can provide more resources to aid in Hawaiian nomenclature:  

- USGS gnis search: http://geonames.usgs.gov/apex/f?p=136:1:0::NO::P1_COUNTY%2CP1_COUNTY_ALONG:n%2C (server down)
- Hawaii State Highways: https://en.wikipedia.org/wiki/List_of_Hawaii_state_highways 
- Hawaiian landmarks http://ulukau.org/elib/cgi-bin/library?e=d-0pepn-000Sec--11haw-50-20-frameset-book--1-010escapewin&a=d&d=D0.2&toc=0
- [S.939 - Hawaiian National Park Language Correction Act of 2000](https://www.congress.gov/106/bills/s939/BILLS-106s939es.pdf)
- [Documentation for ISO 639 identifier: haw](http://www-01.sil.org/iso639-3/documentation.asp?id=haw)
- [Pūnana Leo](https://en.wikipedia.org/wiki/P%C5%ABnana_Leo)
- [Hawaiian Language wikipedia](https://en.wikipedia.org/wiki/Hawaiian_language)

Hawaii OSM project wiki: 
- http://wiki.openstreetmap.org/wiki/Hawaii